## Taksk 1
Create a function which calculates and returns the FIR filter **coefficients** analytically (= using **sinc** functions) for a **combined highpass and bandstop** filter. 
* The function should automatically decide how many coefficients are required. 
* The function arguments should be (a) the **sampling rate** and (b) the **cutoff frequencies**. 
* Decide which **cutoff frequencies** are needed and provide explanations by referring to the **spectra and/or fundamental frequencies** of the ECG

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def combined_highpass_bandstop_fir(sampling_rate, hp_cutoff=0.5, bs_center=50, bs_bandwidth=2):
    
    # Define Nyquist frequency and normalized cutoff
    nyquist = sampling_rate / 2
    normalized_hp_cutoff = hp_cutoff / nyquist
    
    # Calculate the high-pass filter order
    hp_transition_band = 0.5  # Example transition bandwidth
    hp_order = int(hp_transition_band / (normalized_hp_cutoff))  # Using Bt/N = (Ws - Wp) formula
    if hp_order % 2 == 0:  # Ensure order is odd for symmetry
        hp_order += 1
    highpass_coeffs = np.sinc(2 * normalized_hp_cutoff * (np.arange(hp_order) - (hp_order - 1) / 2))
    highpass_coeffs *= np.hamming(hp_order)
    highpass_coeffs = -highpass_coeffs
    highpass_coeffs[(hp_order - 1) // 2] += 1  # Make it a high-pass filter

    # Calculate the band-stop filter order
    bs_transition_band = 0.5  # Example transition bandwidth for the band-stop filter
    normalized_bs_low_cutoff = (bs_center - bs_bandwidth / 2) / nyquist
    normalized_bs_high_cutoff = (bs_center + bs_bandwidth / 2) / nyquist
    bs_order = int(bs_transition_band / (normalized_bs_high_cutoff - normalized_bs_low_cutoff))
    if bs_order % 2 == 0:
        bs_order += 1
    bandstop_coeffs = np.sinc(2 * normalized_bs_high_cutoff * (np.arange(bs_order) - (bs_order - 1) / 2)) \
                      - np.sinc(2 * normalized_bs_low_cutoff * (np.arange(bs_order) - (bs_order - 1) / 2))
    bandstop_coeffs *= np.hamming(bs_order)
    bandstop_coeffs[(bs_order - 1) // 2] += 1

    # Combine high-pass and band-stop filter coefficients by convolution
    combined_coeffs = np.convolve(highpass_coeffs, bandstop_coeffs)
    combined_coeffs /= np.sum(combined_coeffs)  # Normalize filter coefficients
    
    return combined_coeffs

## Task 2
Create an efficient Python FIR filter class which implements an FIR filter and has a method of the form value dofilter(self,value) where both the value argument and return value are scalars and not vectors (!) so that it can be used in a real-time system. The constructor of the class takes the coefficients as its input: 

>class FIRfilter: \
>def __init__(self,_coefficients): \
>your code here \
>def dofilter(self,v): \
>your code here \
>return result 

Filter both ECG recordings with the above FIR filter class using the coefficients from 1. Simulate real-time processing by feeding the ECGs sample by sample into your FIR filter class. Make sure that the ECGs look intact and that they are not distorted (PQRST intact). Provide appropriate plots in a vector-graphics format.

In [ ]:
path = 'ECG Traces-20241104/Noisy ECGs/Noisy_ECG_1000Hz_1.dat'
sampling_rate = 1000

def load_ecg_signal(filename, sampling_rate):
    try:
        ecg_signal = np.loadtxt(filename)
        return ecg_signal
    except ValueError:
        print('Error: Could not load file as text')
        return None
    

# def apply_filter(ecg_signal, sampling_rate):
#     fir_coeffs = combined_highpass_bandstop_fir(sampling_rate)  # Generate the FIR filter coefficients
#     filtered_ecg_signal = np.convolve(ecg_signal, fir_coeffs, mode='same') # Apply the filter using convolution
    
#     return filtered_ecg_signal

# # Load ECG signal data from file
# ecg_signal = load_ecg_signal(filename, sampling_rate)

# # Apply the combined filter
# filtered_ecg_signal = apply_filter(ecg_signal, sampling_rate)


class FIRfilter:
    def __init__(self, coefficients):
        self.coefficients = coefficients
        self.buffer = np.zeros(len(coefficients))

    def dofilter(self, value):
        self.buffer = np.roll(self.buffer, -1)
        self.buffer[-1] = value
        result = np.dot(self.coefficients, self.buffer)
        return result

# Create FIR filter coefficients using the function from Task 1
fir_coeffs = combined_highpass_bandstop_fir(sampling_rate)

# Initialize FIR filter with the coefficients
fir_filter = FIRfilter(fir_coeffs)

# Simulate real-time processing by filtering the ECG signal sample by sample
filtered_ecg_signal_realtime = np.array([fir_filter.dofilter(sample) for sample in ecg_signal])

# Plot the original and filtered signals
plt.figure(figsize=(14, 6))
plt.plot(ecg_signal, label='Original ECG Signal', alpha=0.6)
plt.plot(filtered_ecg_signal_realtime, label='Filtered ECG Signal (Real-time)', color='red', linewidth=1)
plt.legend()
plt.title('ECG Signal Before and After Real-time Filtering')
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.show()